<a href="https://colab.research.google.com/github/danielseagrass99-sys/beginning-bioinformatics-/blob/main/Bioinformatics_Module4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TFBS "co-regulation" finder: scan promoters with a PWM (GFF3-only)

!pip -q install biopython pandas

import io, re, math, sys
from typing import Dict, List, Tuple
from collections import defaultdict

import pandas as pd
from google.colab import files
from Bio import SeqIO, motifs
from Bio.Seq import Seq

# ---------------------------
# Settings for the exercise
# ---------------------------
UPSTREAM_BP   = 1000      # length of upstream promoter to scan (5' of TSS, strand-aware)
PSEUDOCOUNTS  = 0.5      # for PFM -> PPM normalization
PCT_OF_MAX    = 0.75     # threshold as % of the max possible PWM score
ABS_THRESHOLD = None     # or set to e.g. 6.0 (log-odds) to use absolute threshold

# ---------------------------
# Upload files
# ---------------------------
print("Upload genome_demo.fasta, genes_demo.gff3, and motif_demo.pfm")
uploaded = files.upload()

def pick(name):
    for k in uploaded:
        if k.endswith(name):
            return k
    return None

genome_fa = pick("genome_demo.fasta") or next(f for f in uploaded if f.lower().endswith((".fa",".fna",".fasta")))
gff3_file = pick("genes_demo.gff3")  or next(f for f in uploaded if f.lower().endswith((".gff3",".gff")))
motif_file= pick("motif_demo.pfm")   or next(f for f in uploaded if f.lower().endswith((".pfm",".jaspar")))

print("Genome:", genome_fa)
print("GFF3  :", gff3_file)
print("Motif :", motif_file)

# ---------------------------
# Helpers
# ---------------------------
def compute_bg_from_sequences(seq_dict):
    counts = defaultdict(int); total = 0
    for s in seq_dict.values():
        for ch in s.upper():
            if ch in "ACGT":
                counts[ch]+=1; total+=1
    if total == 0:
        return {"A":0.25,"C":0.25,"G":0.25,"T":0.25}
    return {b: counts.get(b,0)/total for b in "ACGT"}

def parse_gff3(path):
    feats = []
    with open(path, "r") as fh:
        for line in fh:
            if not line.strip() or line.startswith("#"):
                continue
            parts = line.rstrip("\n").split("\t")
            if len(parts) < 9:
                continue
            seqid, source, ftype, start, end, score, strand, phase, attrs = parts
            if ftype not in ("gene", "mRNA"):
                continue
            try:
                start = int(start); end = int(end)
            except:
                continue
            attrd = {}
            for kv in attrs.split(";"):
                if "=" in kv:
                    k,v = kv.split("=",1); attrd[k]=v
            gene_id = attrd.get("Name") or attrd.get("gene") or attrd.get("gene_name") or attrd.get("locus_tag") or attrd.get("ID") or f"{seqid}:{start}-{end}"
            feats.append({"seqid": seqid, "start": start, "end": end, "strand": strand, "id": gene_id})
    return feats

def clamp(a, lo, hi): return max(lo, min(a, hi))

# ---------------------------
# Load genome & promoters
# ---------------------------
genome = {rec.id: str(rec.seq).upper() for rec in SeqIO.parse(genome_fa, "fasta")}
bg = compute_bg_from_sequences(genome)

features = parse_gff3(gff3_file)
promoters = {}
for f in features:
    chrom = genome.get(f["seqid"])
    if chrom is None:
        continue
    L = len(chrom)
    if f["strand"] == "+":
        tss = f["start"]
        up_end   = clamp(tss-1, 0, L)
        up_start = clamp(up_end - UPSTREAM_BP, 0, L)
        prom = chrom[up_start:up_end]
    else:
        tss = f["end"]
        down_start = clamp(tss, 0, L)
        down_end   = clamp(tss + UPSTREAM_BP, 0, L)
        prom = chrom[down_start:down_end]
        prom = str(Seq(prom).reverse_complement())
    if len(prom) >= 2:   # must be >= motif length; we’ll check later
        promoters[f["id"]] = prom.upper()

print(f"Loaded {len(promoters)} promoters.")
print("Background (A,C,G,T):", bg)

# ---------------------------
# Motif: PFM -> PPM -> PSSM (log-odds)
# ---------------------------
with open(motif_file, "r") as fh:
    m = motifs.read(fh, "jaspar")

ppm = m.counts.normalize(pseudocounts=PSEUDOCOUNTS)         # probabilities
pssm = ppm.log_odds(background=bg)                          # log2-odds

# Biopython’s matrices are row-major (rows = alphabet, cols = positions).
alphabet = getattr(pssm, "alphabet", "ACGT")
row_index = {b:i for i,b in enumerate(alphabet)}
motif_len = len(pssm[0])

PWM = []
for i in range(motif_len):
    PWM.append({
        'A': float(pssm[row_index['A']][i]),
        'C': float(pssm[row_index['C']][i]),
        'G': float(pssm[row_index['G']][i]),
        'T': float(pssm[row_index['T']][i]),
    })

def max_possible_score(pwm):
    return sum(max(col.values()) for col in pwm)

MAX = max_possible_score(PWM)
if ABS_THRESHOLD is None:
    cutoff = PCT_OF_MAX * MAX
    cutoff_desc = f"{PCT_OF_MAX:.0%} of max (={cutoff:.3f})"
else:
    cutoff = float(ABS_THRESHOLD)
    cutoff_desc = f"absolute log-odds ≥ {cutoff:.3f}"

print(f"Motif length: {motif_len}; threshold: {cutoff_desc}")

# ---------------------------
# Scanner (both strands)
# ---------------------------
def score_kmer(kmer, pwm):
    if len(kmer) != len(pwm): return None
    s = 0.0
    for i,b in enumerate(kmer.upper()):
        if b not in "ACGT": return None
        s += pwm[i][b]
    return s

def best_hit(seq, pwm):
    L = len(seq); k = len(pwm)
    best = (-1e18, -1, '+')
    for i in range(L-k+1):
        s = score_kmer(seq[i:i+k], pwm)
        if s is not None and s > best[0]:
            best = (s, i, '+')
    rc = str(Seq(seq).reverse_complement())
    for i in range(L-k+1):
        s = score_kmer(rc[i:i+k], pwm)
        if s is not None and s > best[0]:
            orig_i = L - k - i
            best = (s, orig_i, '-')
    return best

# ---------------------------
# Scan promoters & report genes
# ---------------------------
rows = []
for gene_id, prom in promoters.items():
    if len(prom) < motif_len:
        rows.append(dict(gene_id=gene_id, promoter_len=len(prom), best_score=None, best_pos0=None, best_strand=None, passes_threshold=False))
        continue
    s,i,strand = best_hit(prom, PWM)
    rows.append(dict(gene_id=gene_id, promoter_len=len(prom), best_score=round(s,3), best_pos0=i, best_strand=strand, passes_threshold=bool(s>=cutoff)))

df = pd.DataFrame(rows).sort_values(["passes_threshold","best_score"], ascending=[False, False])
df.to_csv("tfbs_hits.csv", index=False)

print("\nTop hits:")
display(df[df.passes_threshold].head(10))
print("\nSaved: tfbs_hits.csv")
files.download("tfbs_hits.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.6 MB/s eta 0:00:00
Upload genome_demo.fasta, genes_demo.gff3, and motif_demo.pfm


In [ ]:
# Frequent Words with Mismatches and Reverse Complements (Rosalind)
# Colab version: upload a file, parse it, run, and print results.

from collections import defaultdict
from typing import Set, Dict, Tuple
import re

# -----------------------
# Core helper functions
# -----------------------
def reverse_complement(s: str) -> str:
    comp = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(comp[b] for b in reversed(s))

def hamming_distance(a: str, b: str) -> int:
    return sum(1 for x, y in zip(a, b) if x != y)

def neighbors(pattern: str, d: int) -> Set[str]:
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return {'A', 'C', 'G', 'T'}
    suffix = pattern[1:]
    suffix_neighbors = neighbors(suffix, d)
    result = set()
    for x in suffix_neighbors:
        if hamming_distance(suffix, x) < d:
            for b in ('A', 'C', 'G', 'T'):
                result.add(b + x)
        else:
            result.add(pattern[0] + x)
    return result

def approx_count(Text: str, pattern: str, d: int) -> int:
    k = len(pattern)
    n = len(Text)
    cnt = 0
    for i in range(n - k + 1):
        if hamming_distance(Text[i:i+k], pattern) <= d:
            cnt += 1
    return cnt

def most_frequent_with_mismatches_and_rc(Text: str, k: int, d: int):
    """
    Returns (answer_set, best_score) where answer_set is the set of k-mers
    that maximize Count_d(Text, p) + Count_d(Text, rc(p)).
    """
    n = len(Text)
    candidates: Set[str] = set()

    # Candidate pool: d-neighborhoods of each window and of its reverse complement
    for i in range(n - k + 1):
        kmer = Text[i:i+k]
        rc_kmer = reverse_complement(kmer)
        candidates |= neighbors(kmer, d)
        candidates |= neighbors(rc_kmer, d)

    best = 0
    scores: Dict[str, int] = defaultdict(int)
    seen_pairs: Set[str] = set()

    for p in candidates:
        rc = reverse_complement(p)
        canonical = min(p, rc)
        if canonical in seen_pairs:
            continue
        seen_pairs.add(canonical)

        c1 = approx_count(Text, p, d)
        if p == rc:
            total = c1  # palindromic, don't double-count
        else:
            c2 = approx_count(Text, rc, d)
            total = c1 + c2

        scores[p] = total
        scores[rc] = total
        if total > best:
            best = total

    winners = {pat for pat, val in scores.items() if val == best}
    return winners, best

# -----------------------
# Input parsing helpers
# -----------------------
def parse_rosalind_input(raw: str) -> Tuple[str, int, int]:
    """
    Robust parser for Rosalind-style input:
    - First non-empty, non-whitespace-only line is Text (DNA).
    - Next line containing two integers is k and d.
    Ignores blank lines and trailing spaces. Works if the 2nd line has extra text.
    """
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip() != ""]
    if not lines:
        raise ValueError("Input file appears empty.")

    # First line with only A/C/G/T (optionally N) is Text
    # (Rosalind problems typically use A/C/G/T only; we accept that.)
    text_line = lines[0]
    if not re.fullmatch(r"[ACGTacgt]+", text_line):
        raise ValueError("First non-empty line must be the DNA Text (A/C/G/T only).")
    Text = text_line.upper()

    # Find a line with two integers (k and d)
    k = d = None
    for ln in lines[1:]:
        nums = re.findall(r"\d+", ln)
        if len(nums) >= 2:  # take the first two integers found
            k, d = int(nums[0]), int(nums[1])
            break
    if k is None or d is None:
        raise ValueError("Could not find a line with two integers (k and d).")

    return Text, k, d

# -----------------------
# Colab upload & driver
# -----------------------
def run_from_uploaded_file():
    """
    In Colab:
      1) Prompts you to upload your Rosalind dataset file.
      2) Parses it.
      3) Runs the solver.
      4) Prints the winners and details.
    """
    try:
        from google.colab import files  # Only available in Colab
    except Exception as e:
        print("Note: google.colab not available. This upload dialog works only in Colab.")
        raise

    print("Please choose your Rosalind input file (e.g., 'dataset_XX_XX.txt')...")
    uploaded = files.upload()  # Shows the upload dialog
    if not uploaded:
        print("No file uploaded.")
        return

    # Take the first uploaded file
    fname = next(iter(uploaded.keys()))
    print(f"\nUploaded: {fname}")

    raw = uploaded[fname].decode("utf-8", errors="replace")
    Text, k, d = parse_rosalind_input(raw)

    print("\n=== Parsed Input ===")
    print(f"Length(Text) = {len(Text)}")
    print(f"k = {k}, d = {d}")
    print(f"Total k-mer windows = {len(Text) - k + 1}")

    winners, best = most_frequent_with_mismatches_and_rc(Text, k, d)

    print("\n=== Result ===")
    print(f"Max total approximate count = {best}")
    print(f"# of winning patterns = {len(winners)}")
    print("Winning k-mers (space-separated, sorted):")
    print(" ".join(sorted(winners)))

    # Optional: show per-winner details (counts on pattern and its RC)
    print("\nDetails per winner:")
    for p in sorted(winners):
        rc = reverse_complement(p)
        c1 = approx_count(Text, p, d)
        c2 = 0 if p == rc else approx_count(Text, rc, d)
        total = c1 if p == rc else c1 + c2
        print(f"  {p}  rc={rc}  Count_d(Text,p)={c1}  Count_d(Text,rc)={c2}  total={total}")

# -----------------------
# Run: open the Colab upload dialog
# -----------------------